In [13]:
import pandas as pd
from haversine import haversine
import numpy as np
import json
import datetime

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [3]:
BL_stores=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/MattressFirm/BL_closed stores to closing mattress firm 208 stores_JL_2018-10-05.csv",dtype=str)
BL_stores=BL_stores[['BL_location_id','BL_latitude_meas','BL_longitude_meas']].drop_duplicates()
BL_stores=BL_stores.rename(columns={"BL_location_id":"location_id"})
BL_stores['BL_latitude_meas']=BL_stores['BL_latitude_meas'].astype(float)
BL_stores['BL_longitude_meas']=BL_stores['BL_longitude_meas'].astype(float)
BL_stores=BL_stores.reset_index()
del BL_stores['index']

In [4]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_level_zips=store_level_zips[(store_level_zips['location_id'].isin(BL_stores['location_id'])) & (store_level_zips['revenue_flag'].isin(['P','S']))]
store_level_zips=store_level_zips[['location_id','zip','revenue_flag']]

In [5]:
BL_stores_1=BL_stores[BL_stores['location_id'].isin(store_level_zips['location_id'])]
BL_stores_2=BL_stores[~BL_stores['location_id'].isin(store_level_zips['location_id'])]

BL_stores_1=pd.merge(BL_stores_1,store_level_zips,on="location_id",how="outer")

BL_stores_1=BL_stores_1.reset_index()
del BL_stores_1['index']

BL_stores_2=BL_stores_2.reset_index()
del BL_stores_2['index']

In [6]:
BL_stores_1['distance']=np.nan
for i in range(len(BL_stores_1)):
    
    store_center=(BL_stores_1['BL_latitude_meas'][i],BL_stores_1['BL_longitude_meas'][i])
    
    if BL_stores_1['zip'][i] in zip_centers.keys():
        zip_lat_long=zip_centers[BL_stores_1['zip'][i]]
        dist=haversine(store_center,zip_lat_long,miles=True)
    else:
        print(BL_stores_1['zip'][i])
        dist=np.nan
    BL_stores_1['distance'][i]=dist

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


85147
85121
85269
62949
27906
29486
76099
77476
22402


In [7]:
BL_stores_1_P=BL_stores_1[BL_stores_1['revenue_flag']=="P"]
BL_stores_1_S=BL_stores_1[BL_stores_1['revenue_flag']=="S"]
BL_stores_1_S=BL_stores_1_S[BL_stores_1_S['distance']<=20]

In [8]:
k=0
df_all_zips_in_10=pd.DataFrame()
for i in range(len(BL_stores)):
    store=BL_stores['location_id'][i]
    BL_lat=BL_stores['BL_latitude_meas'][i]
    BL_lng=BL_stores['BL_longitude_meas'][i]
    store_center=(BL_lat,BL_lng)
    for zip_with_center in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_with_center],miles=True)
        if dist<=10:
            k+=1
            df=pd.DataFrame({"location_id":store,"BL_latitude_meas":BL_lat,"BL_longitude_meas":BL_lng,
                            "zip":zip_with_center,"revenue_flag":"zips_in_10","distance":dist},index=[k])
            df_all_zips_in_10=df_all_zips_in_10.append(df)
            
    

In [9]:
output=BL_stores_1_P.append(BL_stores_1_S).append(df_all_zips_in_10)
output_dedup=output.sort_values(['revenue_flag'],ascending=True).reset_index()
del output_dedup['index']
output_dedup=output_dedup.drop_duplicates(['zip'])

In [14]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/MattressFirm/BL_zips_for_nearest_BL_from_closing_MF_stores_JL"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_dedup.to_excel(writer,"unique_zips",index=False)
output=output.sort_values(['location_id','revenue_flag'])
output.to_excel(writer,"zips_by_store",index=False)
writer.save()

